In [ ]:
#Runs Game of Life in a pygame interface. The user can draw intitial conditions. Press R to reset.
# Pygame code inspired by GABE_EDD - https://www.reddit.com/r/pygame/comments/11aiie6/john_conways_game_of_life/
import pygame
import sys
import numpy as np


pygame.init()

#Screen updates per second. Smooth whilst also showing steps clearly
FPS = 6

#Defining Colours to be used
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
GREY = (200, 200, 200)
GREEN = (0, 200, 0)
RED = (200, 0, 0)

#Screen Setup - size, time
screen = pygame.display.set_mode((500, 300))
pygame.display.set_caption("Conway's Game of Life - Setup")
clock = pygame.time.Clock()

font = pygame.font.SysFont(None, 36)
grid_size_input = ''
input_active = True
running = False


def draw_input_screen(surface, input_text):
    #Function to draw input screen
    surface.fill(WHITE)
    prompt_text = font.render("Enter grid size (e.g., 8 for 8x8):", True, BLACK)
    surface.blit(prompt_text, (50, 50))
    input_box = pygame.Rect(50, 100, 200, 50)
    pygame.draw.rect(surface, GREY, input_box)
    text_surface = font.render(input_text, True, BLACK)
    surface.blit(text_surface, (input_box.x + 10, input_box.y + 10))
    pygame.display.flip()

# Input loop
while input_active:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_RETURN and grid_size_input.isdigit():
                #takes userinput from menu screen.
                GRID_SIZE = int(grid_size_input)
                input_active = False
            elif event.key == pygame.K_BACKSPACE:
                grid_size_input = grid_size_input[:-1]
            elif event.unicode.isdigit():
                grid_size_input += event.unicode
    draw_input_screen(screen, grid_size_input)
    clock.tick(FPS)

#Screen Parameters
#Tile size must be integer else recieve boolean error.
SCREEN_SIZE = 600
TILE_SIZE = int(SCREEN_SIZE/GRID_SIZE)
screen = pygame.display.set_mode((SCREEN_SIZE, SCREEN_SIZE + 50))
pygame.display.set_caption("Conway's Game of Life")

grid = np.zeros((GRID_SIZE, GRID_SIZE), dtype=int)


def draw_grid(surface, grid):
    #Draw grid. Size of page is fixed.
    surface.fill(WHITE)
    for row in range(GRID_SIZE):
        for col in range(GRID_SIZE):
            #make live cells (1) black and dead cells (0) white
            color = BLACK if grid[row, col] == 1 else WHITE
            pygame.draw.rect(surface, color, (col * TILE_SIZE, row * TILE_SIZE, TILE_SIZE, TILE_SIZE))
            #Draw gridlines in grey
            pygame.draw.rect(surface, GREY, (col * TILE_SIZE, row * TILE_SIZE, TILE_SIZE, TILE_SIZE), 1)

def update_grid(grid):
    #Conway's Original Rule set
    new_grid = np.copy(grid)
    for row in range(GRID_SIZE):
        for col in range(GRID_SIZE):
            #Calcualte number of neighbours for each tile. Ensure grid slicing does not go out of bounds with max/min. 
            #End index is exclusive, so col+2 is used etc.
            neighbours = np.sum(grid[max(0, row-1):min(GRID_SIZE, row+2), max(0, col-1):min(GRID_SIZE, col+2)]) - grid[row, col]
            
            #Conway's Genetic laws
            
            #Over/Under Population - death
            if grid[row, col] == 1 and (neighbours < 2 or neighbours > 3):
                new_grid[row, col] = 0
            
            #Reproduction 
            elif grid[row, col] == 0 and neighbours == 3:
                new_grid[row, col] = 1
    
    return new_grid


def draw_start_button(surface):
    
    #Positions the button. Arguments for co-ords(x,y) and button size(x,y)
    button_rect = pygame.Rect(10, SCREEN_SIZE + 10, 100, 30)
    
    #Draws coloured rectangle that text is on.
    pygame.draw.rect(surface, GREEN if not running else RED, button_rect)
    text = font.render("Start" if not running else "Stop", True, WHITE)
    
    #Draw text on button with edge buffer
    surface.blit(text, (button_rect.x + 20, button_rect.y + 5))
    return button_rect


#Main loop
while True:
    #Iterate over events in queue
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
        #Actions on mouse click
        elif event.type == pygame.MOUSEBUTTONDOWN:
            #Store position of click
            x, y = pygame.mouse.get_pos()
            #User draws the starting condition if within the grid
            if y < SCREEN_SIZE and not running:
                col, row = x // TILE_SIZE, y // TILE_SIZE
                grid[row, col] = 0 if grid[row, col] == 1 else 1
            #Outside the grid, checks if mouse hit the start button
            elif y >= SCREEN_SIZE:
                button_rect = draw_start_button(screen)
                if button_rect.collidepoint(x, y):
                    running = not running
        #Key Press Commands - 'r' to reset the grid back to empty.
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_r:
                grid = np.zeros((GRID_SIZE, GRID_SIZE), dtype=int)
                running = False

    if running:
        grid = update_grid(grid)

    draw_grid(screen, grid)
    draw_start_button(screen)
    pygame.display.flip()
    clock.tick(FPS)

pygame 2.6.1 (SDL 2.28.4, Python 3.12.3)
Hello from the pygame community. https://www.pygame.org/contribute.html
